In [5]:
from collections import deque
import requests
from bs4 import BeautifulSoup


## Collect URLs

In [19]:

def GetAllinks(url, urls, d):
    q=[(url,0)]
    while q :
        url,currDept=q.pop(0)
        # add_row={'url':url}
        urls.append(url)
        # print(currDept)
        if currDept==d:
            continue
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')
        for link in soup.find_all('a'):
            gotLink = link.get('href')
            
            if(gotLink !=None):
                if 'http' in gotLink:
                    if 'python' in gotLink:
                        if gotLink not in urls:
                            q.append((gotLink,currDept+1))
                   
                else:
                    q.append((url+gotLink,currDept+1))


In [33]:
def GetAllinks2(url, urls, d):
    q = [(url, 0)]
    visited_urls = set()

    while q:
        url, currDept = q.pop(0)

        # Check if the URL has already been visited
        if url in visited_urls:
            continue

        visited_urls.add(url)
        urls.append(url)
        # print(currDept)

        if currDept == d:
            continue

        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'html.parser')

        for link in soup.find_all('a'):
            gotLink = link.get('href')

            if gotLink is not None:
                if 'http' in gotLink and 'python' in gotLink:
                    if gotLink not in visited_urls:
                        q.append((gotLink, currDept + 1))
                else:
                    q.append((url + gotLink, currDept + 1))

In [26]:
start_url="https://www.geeksforgeeks.org/python3-tutorial/"
links=[]
GetAllinks(start_url,links,2)

### removing duplicate links

In [27]:
print(len(links))
unique_links = list(set(links))
print(len(unique_links))
# add url tag to url
unique_url_links=[]
for link in unique_links:
    unique_url_links.append({'url':link})
print(len(unique_url_links),unique_url_links[0])

25844
4990
4990 {'url': 'https://www.geeksforgeeks.org/python-list-exercise/?ref=dhm#'}


In [82]:
# add title to links
# import requests
# from bs4 import BeautifulSoup

# def GetAllinks(url, urls, d):
#     q = [(url, 0)]

#     while q:
#         url, currDept = q.pop(0)
#         add_to_array={'url':url,
#                       'title':GetPageTitle(url)
#                       }
#         urls.append(add_to_array)  # Append page title along with the URL
#         print(currDept)
#         if len(urls)>10:
#             return
#         if currDept == d:
#             continue

#         reqs = requests.get(url)
#         soup = BeautifulSoup(reqs.text, 'html.parser')

#         for link in soup.find_all('a'):
#             gotLink = link.get('href')

#             if gotLink is not None:
#                 if 'http' in gotLink:
#                     if 'python' in gotLink:
#                         q.append((gotLink, currDept + 1))
#                 else:
#                     q.append((url + gotLink, currDept + 1))

# def GetPageTitle(url):
#     try:
#         reqs = requests.get(url)
#         soup = BeautifulSoup(reqs.text, 'html.parser')
#         title = soup.title.string
#         return title
#     except Exception as e:
#         print(f"Error fetching page title for {url}: {e}")
#         return "Page title not available"

# # Example usage:
# urls = []
# GetAllinks(start_url, urls, 1)
# for url in urls:
#     print(url)


In [85]:
links[3600]

'https://www.geeksforgeeks.org/python3-if-if-else-nested-if-if-elif-statements/?ref=dhm'

## storing json file

In [4]:
import json
import os
import pandas as pd

In [14]:
def store_json(content, path, file_name):
    def add_url_tag(myset):
        # mylist=list(myset)
        save=[]
        for link in myset:
            save.append({'link':link})
        return save
    content=add_url_tag(content)
    try:
        # Ensure the specified directory exists
        os.makedirs(path, exist_ok=True)

        # Store the content in a JSON file
        with open(os.path.join(path, file_name), 'w') as json_file:
            json.dump(content, json_file)

        print(f'The content has been stored in the JSON file: {file_name}')
    except Exception as e:
        print(f'Error storing JSON content: {e}')

In [13]:
store_json(unique_url_links,path='new_data',file_name='geeks_for_geeks_depth5.json')
unique_url_links[len(unique_url_links)-1]

NameError: name 'store_json' is not defined

## Only considering unique links for crwaling

In [2]:
def GetAllinks_set(url, urls_set, d):
    q=[(url,0)]
    aa,bb=1,1
    while q :
        try:
            url,currDept=q.pop(0)
            # add_row={'url':url}
            if url in urls_set:
                continue
            urls_set.add(url)
            # print(currDept)
            if currDept==d:
                continue
            reqs = requests.get(url)
            soup = BeautifulSoup(reqs.text, 'html.parser')
            for link in soup.find_all('a'):
                try:
                    gotLink = link.get('href')
                    
                    if(gotLink !=None):
                        if 'http' in gotLink:
                            if 'python' in gotLink:
                                # if gotLink not in urls:
                                q.append((gotLink,currDept+1))
                        
                        else:
                            q.append((url+gotLink,currDept+1))
                except KeyboardInterrupt:
                    print("Inner for loop error :",aa)
                    aa+=1
                    continue
        except (requests.exceptions.RequestException,KeyboardInterrupt) as e:
            print("outer while loop error : ",bb," : ",e)
            bb+=1
            return urls_set

In [42]:
def GetAllinks_set2(url, urls_set, d):
    q = [(url, 0)]
    collected_data = set()

    while q:
        url, currDept = q.pop(0)

        # Check if the URL has already been visited
        if url in urls_set:
            continue

        urls_set.add(url)
        collected_data.add(url)  # Save the data

        if currDept == d:
            continue

        try:
            reqs = requests.get(url)
            reqs.raise_for_status()  # Check for HTTP errors

            soup = BeautifulSoup(reqs.text, 'html.parser')

            for link in soup.find_all('a'):
                gotLink = link.get('href')

                if gotLink is not None:
                    if 'http' in gotLink:
                        if 'python' in gotLink:
                            if gotLink not in urls_set:
                                q.append((gotLink, currDept + 1))
                    else:
                        q.append((url + gotLink, currDept + 1))

        except (requests.exceptions.RequestException,KeyboardInterrupt) as e :
            print(f"Connection error: {e}")
            return collected_data  # End the function and return the collected data

    return collected_data  # Return the collected data if the function completes successfully


In [ ]:
links2=set()
start_url="https://www.geeksforgeeks.org/python3-tutorial/"
links_result=GetAllinks_set(start_url,links2,3)
# print("Returned =",len(links_result))
print(len(links2))
links2
store_json(links2,path='new_data',file_name='geeks_for_geeks_depth5_new.json')
df=pd.read_json('./new_data/geeks_for_geeks_depth5_new.json')
len(df)

In [17]:
df2=pd.read_csv('./python_tutorial_links.txt', header=None)
link_set=[]
for link in df2:
    link_set.append(df2[link][0])
    # print(link)
len(link_set)
store_json(link_set,path='new_data/starting_urls',file_name='python_tutorial.json')

The content has been stored in the JSON file: python_tutorial.json


In [18]:
def add_to_set(path,get_label):
    res=set()
    data=pd.read_json(path)
    # print(data['link'])
    for index,row in data.iterrows():
        res.add(row[get_label])
    return res
d3=add_to_set('./new_data/geeks_for_geeks_d3_41k.json','link')
d2=add_to_set('./new_data/geeks_for_geeks_depth2.json','url')
d5=add_to_set('./new_data/geeks_for_geeks_depth5.json','link')
d5n=add_to_set('./new_data/geeks_for_geeks_depth5_new.json','link')
print(len(d3),len(d2),len(d5),len(d5n))
d23=d3.union(d2)
d235=d23.union(d5)
d235n=d235.union(d5n)
len(d235n)

40957 4990 4990 5972


40961

The content has been stored in the JSON file: geeks_for_geeks41k_final.json
